In [1]:
# Package installation
!pip install sklearn
!pip install turicreate

     |████████████████████████████████| 92.0MB 66kB/s 
     |████████████████████████████████| 3.4MB 47.7MB/s 
     |████████████████████████████████| 327kB 42.5MB/s 
     |████████████████████████████████| 86.4MB 75kB/s 
     |████████████████████████████████| 51kB 5.8MB/s 
     |████████████████████████████████| 3.8MB 42.8MB/s 
     |████████████████████████████████| 450kB 45.0MB/s 
  Created wheel for resampy: filename=resampy-0.2.1-cp36-none-any.whl size=320848 sha256=371229177c31cc98eac931d15d4f2d23379187a70b89597e925553e592816d42
  Stored in directory: /root/.cache/pip/wheels/ff/4f/ed/2e6c676c23efe5394bb40ade50662e90eb46e29b48324c5f9b
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=5f0a31764c1caf848456c9d11b115229f3837ea914b514aa994be3ef4278bb5f
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built resampy gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, 

In [2]:
# Import dependencies
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
import turicreate as tc
import turicreate.aggregate as agg

# **Input Data**

In [12]:
# Import drive first before exporting as csv file
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [15]:
# Use pandas to read into data
df = pd.read_csv('gdrive/My Drive/bq-iowa-liquor/bq-iowa-liquor.csv')
print(df.shape)
df.head()

(752641, 3)


,store_number,item_number,bottles_sold
0,3354,38177,7920
1,3447,38177,7920
2,3944,38177,7920
3,3477,38177,7920
4,3524,38177,7632


# **Data Preparation**

In [33]:
# Get rename data for product/customer matrix
df.rename(columns={'store_number': 'CustomerID', \
                   'item_number': 'StockCode', \
                   'bottles_sold': 'Quantity'}, inplace=True)
data = df
print(data.shape)
data.head()

(752641, 3)


,CustomerID,StockCode,Quantity
0,3354,38177,7920
1,3447,38177,7920
2,3944,38177,7920
3,3477,38177,7920
4,3524,38177,7632


**Create Data with User, Product and Purchase Count**

In [34]:
# Group by to get purchase quantity by customer ID and stock code
data = data.groupby(['CustomerID','StockCode']) \
    .agg({'Quantity':'sum'}) \
    .reset_index()
print(data.shape)
data.head()

(446139, 3)


,CustomerID,StockCode,Quantity
0,2106,1799,42
1,2106,4084,6
2,2106,4867,12
3,2106,5006,1
4,2106,5036,3


In [35]:
# Check to ensure no negative purchase order quantity
data[data['Quantity']<0]

,CustomerID,StockCode,Quantity


In [36]:
# Check data types for current dataframe. CustomerID and StockCode must be string
data.dtypes

CustomerID    int64
StockCode     int64
Quantity      int64
dtype: object

*Outcome: Need to change CustomerID and StockCode from integer to string*

In [37]:
# Format CustomerID & StockCode from integer to string
data[['CustomerID','StockCode']] = data[['CustomerID','StockCode']].astype(int).astype(str)
data.dtypes

CustomerID    object
StockCode     object
Quantity       int64
dtype: object

**Create Dummy Dataset**

In [43]:
# Create dummy
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy
data_dummy = create_data_dummy(data)
print(data_dummy.shape)
data_dummy.head()

(446139, 4)


,CustomerID,StockCode,Quantity,purchase_dummy
0,2106,1799,42,1
1,2106,4084,6,1
2,2106,4867,12,1
3,2106,5006,1,1
4,2106,5036,3,1


**Normalize Item Values Across Users**

In [40]:
# Function to normalize item values across users
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='Quantity', index='CustomerID', columns='StockCode')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['CustomerID'], value_name='scaled_purchase_freq').dropna()

In [51]:
# Execute function
norm_data = normalize_data(data)
print(norm_data.shape)
norm_data.head()

(440512, 3)


,CustomerID,StockCode,scaled_purchase_freq
18,2502,100015,1.000000
20,2506,100015,0.285714
27,2515,100015,1.000000
35,2528,100015,0.571429
39,2536,100015,1.000000


# **Split Train and Test Datasets**

In [47]:
# Declare function to split train and test data
def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [48]:
# Split normalised scaled frequency data
train_data_norm, test_data_norm = split_data(norm_data)

In [49]:
# Split dummy data
train_data_dummy, test_data_dummy = split_data(data_dummy)

In [50]:
# Split purchase count data
train_data, test_data = split_data(data)

# **Define Models Using Turicreate Library**

In [52]:
# Define variables for field names
user_id = 'CustomerID'
item_id = 'StockCode'
users_to_recommend = list(data['CustomerID'])
n_rec = 10 # number of items to recommend
n_display = 30 # to display the first few rows in an output dataset

In [53]:
# Declare function for all models
def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

# **Popularity Model as Baseline**

**Model based on Purchase Count**

In [54]:
# Using Purchase Count
name = 'popularity'
target = 'Quantity'
popularity = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 356911 observations with 1840 users and 3711 items.

Data prepared in: 0.462767s

356911 observations to process; with 3711 unique items.

recommendations finished on 1000/446139 queries. users per second: 24876.2

...

recommendations finished on 443000/446139 queries. users per second: 21510.2

recommendations finished on 444000/446139 queries. users per second: 21499.3

recommendations finished on 445000/446139 queries. users per second: 21476.2

recommendations finished on 446000/446139 queries. users per second: 21481.2

+------------+-----------+---------+------+
| CustomerID | StockCode |  score  | rank |
+------------+-----------+---------+------+
|    2106    |   936600  | 14760.0 |  1   |
|    2106    |   962094  | 14580.0 |  2   |
|    2106    |   908806  |  3024.0 |  3   |
|    2106    |   989289  |  2912.0 |  4   |
|    2106    |   962093  |  2700.0 |  5   |
|    2106    |   989291  |  1632.0 |  6   |
|    2106    |   989290  |  1584.0 |  7   |
|    2106    |   987514  |  1152.0 |  8   |
|    2106    |   944650  |  960.0  |  9   |
|    2106    |   930746  |  960.0  |  10  |
|    2106    |   936600  | 14760.0 |  1   |
|    2106    |   962094  | 14580.0 |  2   |
|    2106    |   908806  |  3024.0 |  3   |
|    2106    |   989289  |  2912.0 |  4   |
|    2106    |   962093  |  2700.0 |  5   |
|    2106    |   989291  |  1632.0 |  6   |
|    2106    |   989290  |  1584.0 |  7   |
|    2106    |   987514  |  1152.0 |  8   |
|    2106    |   944650  |  960.0  |  9   |
|    2106    |   930746  |  960.

**Model based on Purchase Dummy**

In [55]:
# Using purchase dummy
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns Quantity;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 356911 observations with 1841 users and 3725 items.

Data prepared in: 0.509151s

...

recommendations finished on 443000/446139 queries. users per second: 21787.5

recommendations finished on 444000/446139 queries. users per second: 21790.8

recommendations finished on 445000/446139 queries. users per second: 21796.2

recommendations finished on 446000/446139 queries. users per second: 21802.5

+------------+-----------+-------+------+
| CustomerID | StockCode | score | rank |
+------------+-----------+-------+------+
|    2106    |   65793   |  1.0  |  1   |
|    2106    |   34003   |  1.0  |  2   |
|    2106    |   65536   |  1.0  |  3   |
|    2106    |   87408   |  1.0  |  4   |
|    2106    |   34155   |  1.0  |  5   |
|    2106    |   48105   |  1.0  |  6   |
|    2106    |   36887   |  1.0  |  7   |
|    2106    |   64489   |  1.0  |  8   |
|    2106    |   36873   |  1.0  |  9   |
|    2106    |   19094   |  1.0  |  10  |
|    2106    |   65793   |  1.0  |  1   |
|    2106    |   34003   |  1.0  |  2   |
|    2106    |   65536   |  1.0  |  3   |
|    2106    |   87408   |  1.0  |  4   |
|    2106    |   34155   |  1.0  |  5   |
|    2106    |   48105   |  1.0  |  6   |
|    2106    |   36887   |  1.0  |  7   |
|    2106    |   64489   |  1.0  |  8   |
|    2106    |   36873   |  1.0  |  9   |
|    2106    |   19094   |  1.0  |  10  |
|    2106    |   65793   |  1.0  |

**Model based on Scaled Purchase Count**

In [56]:
# Using scaled purchase count
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 352409 observations with 1839 users and 2949 items.

Data prepared in: 0.623681s

352409 observations to process; with 2949 unique items.

recommendations finished on 1000/446139 queries. users per second: 30502.7

...

recommendations finished on 443000/446139 queries. users per second: 24386.9

recommendations finished on 444000/446139 queries. users per second: 24389.8

recommendations finished on 445000/446139 queries. users per second: 24314

recommendations finished on 446000/446139 queries. users per second: 24264.2

+------------+-----------+-------+------+
| CustomerID | StockCode | score | rank |
+------------+-----------+-------+------+
|    2106    |   997878  |  1.0  |  1   |
|    2106    |   958875  |  1.0  |  2   |
|    2106    |   986980  |  1.0  |  3   |
|    2106    |   42400   |  1.0  |  4   |
|    2106    |   36880   |  1.0  |  5   |
|    2106    |   994972  |  1.0  |  6   |
|    2106    |   903486  |  1.0  |  7   |
|    2106    |   966123  |  1.0  |  8   |
|    2106    |   75161   |  1.0  |  9   |
|    2106    |   100795  |  1.0  |  10  |
|    2106    |   997878  |  1.0  |  1   |
|    2106    |   958875  |  1.0  |  2   |
|    2106    |   986980  |  1.0  |  3   |
|    2106    |   42400   |  1.0  |  4   |
|    2106    |   36880   |  1.0  |  5   |
|    2106    |   994972  |  1.0  |  6   |
|    2106    |   903486  |  1.0  |  7   |
|    2106    |   966123  |  1.0  |  8   |
|    2106    |   75161   |  1.0  |  9   |
|    2106    |   100795  |  1.0  |  10  |
|    2106    |   997878  |  1.0  |

**Baseline Summary**

In [57]:
train_data.groupby(key_column_names='StockCode', operations={'mean_qty': agg.MEAN('Quantity')}).sort('mean_qty', ascending = False).head(20)

StockCode,mean_qty
936600,14760.0
962094,14580.0
908806,3024.0
989289,2912.0
962093,2700.0
989291,1632.0
989290,1584.0
987514,1152.0
930746,960.0
944650,960.0


# **Collaborative Filtering Model**

## Cosine Similarity

**Model based on Purchase Count**

In [58]:
# Using purchase count
name = 'cosine'
target = 'Quantity'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 356911 observations with 1840 users and 3711 items.

Data prepared in: 0.492562s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 443000/446139 queries. users per second: 3459.78

recommendations finished on 444000/446139 queries. users per second: 3459.26

recommendations finished on 445000/446139 queries. users per second: 3461.48

recommendations finished on 446000/446139 queries. users per second: 3463.87

+------------+-----------+--------------------+------+
| CustomerID | StockCode |       score        | rank |
+------------+-----------+--------------------+------+
|    2106    |   34422   | 5.529014317936628  |  1   |
|    2106    |   42717   | 5.2192492725222355 |  2   |
|    2106    |   65257   |  4.9327594199989   |  3   |
|    2106    |   41705   | 4.408116348691888  |  4   |
|    2106    |   15627   | 4.189573027285077  |  5   |
|    2106    |   28867   | 4.114771939905621  |  6   |
|    2106    |    5486   | 3.430462270169645  |  7   |
|    2106    |   67527   | 2.857576581271919  |  8   |
|    2106    |   89787   | 2.825084089647054  |  9   |
|    2106    |   89154   | 2.652355605319911  |  10  |
|    2106    |   34422   | 5.529014317936628  |  1   |
|    2106    |   42717   | 5.2192492725222355 |  2   |
|    2106    |   65257   |  4.9327594199989   |  3   |
|    2106    |   41705   | 4.408116348691888  |  4   |
|    2106    |   15627   | 4.189573027285077  |  5   |
|    2106 

**Model based on Purchase Dummy**

In [59]:
# Using purchase dummy
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns Quantity;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 356911 observations with 1841 users and 3725 items.

Data prepared in: 0.494146s

...

recommendations finished on 443000/446139 queries. users per second: 3541.95

recommendations finished on 444000/446139 queries. users per second: 3541.71

recommendations finished on 445000/446139 queries. users per second: 3543.8

recommendations finished on 446000/446139 queries. users per second: 3546.16

+------------+-----------+---------------------+------+
| CustomerID | StockCode |        score        | rank |
+------------+-----------+---------------------+------+
|    2106    |   26828   |  0.1311610099596855 |  1   |
|    2106    |   43371   | 0.12987541739757244 |  2   |
|    2106    |   41846   |  0.1134867217296209 |  3   |
|    2106    |   41692   | 0.11297521484203828 |  4   |
|    2106    |   65126   | 0.10386151931224726 |  5   |
|    2106    |   18348   | 0.09727571102289054 |  6   |
|    2106    |   43316   | 0.09648210818950946 |  7   |
|    2106    |   41694   | 0.09186256986397963 |  8   |
|    2106    |   82787   | 0.09106353017000052 |  9   |
|    2106    |   26710   |  0.0900751239214188 |  10  |
|    2106    |   26828   |  0.1311610099596855 |  1   |
|    2106    |   43371   | 0.12987541739757244 |  2   |
|    2106    |   41846   |  0.1134867217296209 |  3   |
|    2106    |   41692   | 0.11297521484203828 |  4   |
|    2106    |   65126   | 0.10386151931224726 |

**Model based on Scaled Purchase Count**

In [ ]:
# Using scaled purchase count
name = 'cosine' 
target = 'scaled_purchase_freq' 
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

## Pearson similarity

**Model based on Purchase Count**

In [ ]:
# Using purchase count
name = 'pearson'
target = 'Quantity'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

**Model based on Purchase Dummy**

In [ ]:
# Using purchase dummy
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

**Model based on Scaled Purchase Count**

In [ ]:
# Using scaled purchase count
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

# **Model Evaluation**

In [ ]:
# Declare initial callable variables for model evaluation
models_w_counts = [popularity, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]
names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

In [ ]:
models_w_counts

In [ ]:
models_w_dummy

In [ ]:
models_w_norm

## Compare all the models based on RMSE and precision-recall characteristics

**Evaluating Purchase Count Models**

In [ ]:
# Using purchase counts models
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)

**Evaluating Purchase Dummy Models**

In [ ]:
# Using dummy models
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)

**Evaluating Scaled Purchase Count Models**

In [ ]:
# Using normalized purchase count models
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

## Evaluation Output

## Evaluation Summary

# **Final Output**